# Benchmarking

We want to compare 14Ma with 0Ma.

Generate ATOM's predictions of what the world looks like and compare those with NASA data and ??something else?? to benchmark ATOM.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import os.path
import numpy as np
import shutil
from pyatom import Model

model = Model()
model.load_config('benchmark.xml')

In [ ]:
rebuild = False
if rebuild:
    # Run the model for 14Ma and 0Ma
    # First, clear output dirs
    for dirname in ['output-0', 'output-14']:
        try:
            shutil.rmtree(dirname)
        except OSError:
            pass

    print dir(model)

    print dir(model.atmosphere)

    print model.atmosphere.output_path
    print model.atmosphere.buoyancy

    # FIXME: there needs to be a trailing slash on output_path; the software should be more tolerant
    model.run(t=0, output_path='output-0/')
    model.run(t=14, output_path='output-14/')


In [ ]:
def plot_xyz(series, title, lats, lons):
    plt.figure(figsize=(15, 8))

    m = Basemap(llcrnrlon=0,llcrnrlat=-80,urcrnrlon=360,urcrnrlat=80,projection='kav7', lon_0=0)
    xi, yi = m(lons, lats)

    cs = m.scatter(xi, yi, marker='.', c=series, alpha=1.0, lw=0)

    # Add Grid Lines
    m.drawparallels(np.arange(-80., 81., 10.), labels=[1,0,0,0], fontsize=10)
    m.drawmeridians(np.arange(-180., 181., 10.), labels=[0,0,0,1], fontsize=10)

    m.drawcoastlines()

    # Add Colorbar
    cbar = m.colorbar(cs, location='bottom', pad="10%")
    # cbar.set_label(tmax_units)

    # Add Title
    plt.title(title)

    plt.show()


In [ ]:
# try it with the XYZ file instead
# print os.path.join(datapath, '[%sMa_Golonka.xyz]_PlotData_Atm.xyz' % Ma)
Ma = 0
datapath = 'output-%s' % Ma

atm_header = ['latitude (deg)', 'longitude (deg)', 'v-velocity (m/s)', 'w-velocity (m/s)', 'temperature (celcius)', 'water_vapour (g/kg)', 'precipitation (mm)', 'precipitable_water (mm)']
hyd_header = ['latitude (deg)', 'longitude (deg)', 'v-velocity (0.724 * x * m/s)', 'w-velocity (0.724 * x * m/s)', 'temperature (273.15 * x - 273.15)', 'salinity (35 * x * psu)', 'bottom_water', 'upwelling', 'downwelling']

adata = np.loadtxt(os.path.join(datapath, '[%sMa_Golonka.xyz]_PlotData_Atm.xyz' % Ma), skiprows=1)
hdata = np.loadtxt(os.path.join(datapath, '[%sMa_Golonka.xyz]_PlotData_Hyd.xyz' % Ma), skiprows=1)

# np.hsplit(rec, 8)
# FIXME: the xyz file says it's lat+lon order, but the range (0-180, 0-360) suggests it's lon/lat. What's the correct answer here?
# lats, lons, vvelocity, wvelocity, temperature, water_vapour, precipitation, precipitable_water = np.hsplit(rec, 8)
# lons, lats, vvelocity, wvelocity, temperature, water_vapour, precipitation, precipitable_water = np.hsplit(rec, 8)



# Plot all Atmosphere outputs

Just the `xyz` files, anyway. There are hundreds of things we could plot...

In [ ]:
# Plot all of Atmosphere outputs

# FIXME: ATOM seems to have lat/lon transposed in the xyz file?
# we also have a mysterious offset
alats = -(adata[:, 1] - 90.)
alons = adata[:, 0]



for index in range(2, 5):
    title = atm_header[index]
    plot_xyz(adata[:, index], title, alats, alons)



In [ ]:
for index in range(5, len(atm_header)):
    title = atm_header[index]
    plot_xyz(adata[:, index], title, alats, alons)



# Plot all Hydrosphere outputs

Just the `xyz` files.

In [ ]:
# Plot all of Hydrosphere outputs

# NOTE: mysterious transpositions
hlats = -(hdata[:, 0] - 90)
hlons = hdata[:, 1]

for index in range(2, 5):
    title = hyd_header[index]
    plot_xyz(hdata[:, index], title, hlats, hlons)


In [ ]:
for index in range(5, 8):
    title = hyd_header[index]
    plot_xyz(hdata[:, index], title, hlats, hlons)


In [ ]:
for index in range(8, len(hyd_header)):
    title = hyd_header[index]
    plot_xyz(hdata[:, index], title, hlats, hlons)



# Compare surface precipitation

In [ ]:
# nsp: NASA Surface Precipitation
nsp = np.genfromtxt('../data/SurfacePrecipitation_NASA.xyz')
# columns lons, lats, precip

plot_xyz(lons=nsp[:, 0], lats=nsp[:, 1], series=nsp[:, 2], title='NASA Surface Precipitation')

plot_xyz(adata[:, 6], 'ATOM Precipitation at Ma=%s' % Ma, alats, alons)



# Compare surface temperature

In [ ]:
# st: Surface Temperature
# I have no idea where that file comes from
st = np.genfromtxt('../data/SurfaceTemperature.xyz')
# columns lons, lats, ?

plot_xyz(lons=st[:, 0], lats=st[:, 1], series=st[:, 2], title='Surface Temperature')
plot_xyz(adata[:, 4], 'ATOM Atmosphere Temperature at Ma=%s' % Ma, alats, alons)
plot_xyz(hdata[:, 4], 'ATOM Hydrosphere Temperature at Ma=%s' % Ma, hlats, hlons)


# Surface Winds benchmarking

Reference data comes from https://podaac.jpl.nasa.gov/dataset/QSCAT_L3_SFC_WIND_SPEED_1DEG_1MO
     

In [ ]:
# load windspeed data
from netCDF4 import Dataset
root = Dataset('windspeed/sfcWind_QuikSCAT_L2B_v20110531_199908-200910.nc')
# print root
# print root.dimensions.keys()
print root.variables.keys()

t = root.variables['time'][-1]  # just take the newest timeslice

# NOTE: wind mags vary substantially with time; what time should we be using? or average it all out?

swlats = root.variables['lat'][:]
swlons = root.variables['lon'][:]
swspeed = root.variables['sfcWind'][-1]

lonv, latv = np.meshgrid(swlons, swlats)

plot_xyz(lats=latv.flatten(), lons=lonv.flatten(), title='Surface Wind QuikSCAT', series=swspeed.flatten())

root.close()

In [ ]:
# load windspeed data
from netCDF4 import Dataset
root = Dataset('windspeed/uas_QuikSCAT_L2B_v20110531_199908-200910.nc')
print root.variables.keys()

t = root.variables['time'][-1]  # just take the newest timeslice

# NOTE: wind mags vary substantially with time; what time should we be using? or average it all out?

swlats = root.variables['lat'][:]
swlons = root.variables['lon'][:]
swspeed = root.variables['uas'][-1]

lonv, latv = np.meshgrid(swlons, swlats)

plot_xyz(lats=latv.flatten(), lons=lonv.flatten(), title='Surface Wind QuikSCAT u-mag', series=swspeed.flatten())

root.close()

In [ ]:
# load windspeed data
from netCDF4 import Dataset
root = Dataset('windspeed/vas_QuikSCAT_L2B_v20110531_199908-200910.nc')
print root.variables.keys()

t = root.variables['time'][-1]  # just take the newest timeslice

# NOTE: wind mags vary substantially with time; what time should we be using? or average it all out?

swlats = root.variables['lat'][:]
swlons = root.variables['lon'][:]
swspeed = root.variables['vas'][-1]

lonv, latv = np.meshgrid(swlons, swlats)

plot_xyz(lats=latv.flatten(), lons=lonv.flatten(), title='Surface Wind QuikSCAT v-mag', series=swspeed.flatten())

root.close()

In [ ]:
for index in range(2, 4):
    title = 'Atmosphere: %s' % atm_header[index]
    plot_xyz(adata[:, index], title, alats, alons)



# Surface Currents

Using data from https://podaac.jpl.nasa.gov/dataset/OSCAR_L4_OC_third-deg


In [ ]:
from netCDF4 import Dataset
root = Dataset('oscar/oscar_vel8879.nc')
print root.variables.keys()

sclats = root.variables['latitude'][:]
sclons = root.variables['longitude'][:]
u = root.variables['u'][0][0][:]
v = root.variables['v'][0][0][:]


print sclats.shape 
print sclons.shape
print u.shape
print v.shape

lonv, latv = np.meshgrid(sclons, sclats)

plot_xyz(lats=latv.flatten(), lons=lonv.flatten(), title='OSCAR Surface Currents u-mag', series=np.log(u.flatten()))


In [ ]:
plot_xyz(lats=latv.flatten(), lons=lonv.flatten(), title='OSCAR Surface Currents v-mag', series=v.flatten())

root.close()

In [ ]:
# NOTE: mysterious transpositions
hlats = -(hdata[:, 0] - 90)
hlons = hdata[:, 1]

for index in range(2, 4):
    title = "Hydrosphere: %s" % hyd_header[index]
    plot_xyz(hdata[:, index], title, hlats, hlons)
